# Analysis of Open Street Maps data

### Area of Interest

I have chosen an area of Southern California specifically Southern Orange County, for the purpose of this analysis. The osm file for this area came in at ~ 75.5mb, so suitable as it meets the project requirements of greater than 50mb.

A map of this area can be obtained from openstreetmap.org utulizing the below coordinates (xml from osm file)

<code><bounds minlat="33.5291000" minlon="-117.8555000" maxlat="33.6594000" maxlon="-117.5640000"/\>'</code>

Below you can see a map of the area selected.

![map_laguna_hills.PNG](https://raw.githubusercontent.com/turner-kevin/Open-Street_Maps/master/images/map_laguna_hills.PNG)

### Auditing Openstreetmap Data

First thing I did was have cursory look at the data in the OSM file by opening it in Notepad++ (a text editor). I noted that the file contained some points of data that may commonly be inconsitently recorded such as street addresses and phone numbers.

One could painstakingly audit each individual line for accuracy (1,091,922 lines of xml), but that would be ludicrous and not meet the expectaions of this project.

So, Python to the rescue!

**NOTE: Code provided by Udacity was written for Python2, however I was determined to complete this project with Python3. A few bits of code needed to be modified for this purpose**

Utilizing code provided by Udacity as general starting point I created three functions to perform auditing programmatically. The program starts by passing a list of tags we want to look closer at. In this scenario we pass <code>audit_elem = {'street', 'zip', 'phone'}</code>

 - Function #1 <code>def audit(audit_elem)</code>simply iterates through the OSM document and passes each XML element to the next function
 
 
 - Function #2 <code>def is_element_to_audit(element, tag):</code> determines if the element contains information that we want to look closer at (ex: Street or Phone). If the element contains information we want to audit, that element is passed to the thrid function
 
 
 - Function #3 <code>def audit_element_type(element, attribute_value)</code> is where the real work happens. Based on the key attribute of the element this functions determines how element should be reviewed (ex: as a Street Address, or a Phone Number) and returns a dictionary of values that don't meet our expected criteria
 
My thought process when writing the three individual functions was for greateer flexibility and extensibility. In fact the heavy lifting is only perfomred in one function, <code>def audit_element_type(element, attribute_value)</code>. If you wanted to audit a another attribute, you only need to write auditing logic into an <code>elif</code> statement and include that element in the <code>audit_elem</code> list.
 
**Full code can be found in the file <code>../scripts/audit_elements.py</code>**

What was the result of the data audit?

Well, lets start with street names. I set out looking for inconsistently abbreviated common street types (ex: Rd for Road, Blvd for Boulevard), but I wasn't exactly sure what I was going to find.

As turns out, there are a lot of streets that have no common street designation at all. This is actually pretty normal for the area though, so we'll accept these values as is. Next, I found standard street names that I would not have considered common before, like 'Glen' (seen in the screenshot below). Again we will accept these values as is.

Ah!, looks like we do have some abbreviated street names in our dataset. We'll be sure to fix those later before creating csv files to import into the SQL database.

All in all we ended up with 8 unique values that needed to be addressed.

<code>{'Ave' : 'Avenue',
'Blvd' : 'Boulevard',
'Ct.' : 'Court',
'Dr' : 'Drive',
'Dr.' : 'Drive',
'Pkwy' : 'Parkway',
'Pkwy.' : 'Parkway',
'Rd' : 'Road'}
</code>

![Streets_Review_03.PNG](https://raw.githubusercontent.com/turner-kevin/Open-Street_Maps/master/images/Streets_Review_03.PNG)

![Streets_Review_04.PNG](https://raw.githubusercontent.com/turner-kevin/Open-Street_Maps/master/images/Streets_Review_04.PNG)

Next I had look at the Zipcode. I specifically only wanted to see the 5 digit zipcodes in the dataset and was unintersted in any addiotnal digits added after a dash (-).

Zipcode data looked pretty clean. A few instances of multiple zipcodes found in one attribute, but they still matched the pattern we were looking for, so we'll leave them as is.

![Zipcodes_Review_01.PNG](https://raw.githubusercontent.com/turner-kevin/Open-Street_Maps/master/images/Zipcodes_Review_01.PNG)

Finally I audited the phone numbers in the daataset.

WOW! the format is all over the place. Some have Country Code prefix, some enclose the area code in parenthesis, then there is muliple sperator values that are in use for spaces to dashes to periods.

I want all phone numbers in a standard +1-111-111-1111 format, so we have some work cut out for us to clean this up, but we will deal with that soon.

![PhoneNumbers_Review_01.PNG](https://raw.githubusercontent.com/turner-kevin/Open-Street_Maps/master/images/PhoneNumbers_Review_01.PNG)

### Fixing Values and Preparing Files SQL Database Load

Since the OSM file is rather large at ~76mb, the first thing I did was create a sample of the data utilizing a script provided.

Script can be found at <code>../scripts/create_sample_file</code>

This script takes every 100 top-level elements from the original OSM file and creates a new file. This new sample file came in at ~7mb, a much more manageble size while testing and validating the next activities.

Much of the validation and csv file creation logic was provided for this project (again I did have to modify it to work with Python3). However I did write the functions used to properly format the data to be writtent to a csv file, as well as, a function to fix the attribute values that we previously identified during the data audit.

As with the auditing functions, I tried to keep it simple and extensible. I chose to write a single function that would handle the heavy lifting of fixing values that needed to be fixed.

There are two primary functions used here.

 - Function #1 <code>def shape_element</code> iterates through the OSM file and determines what should happen with each line. For Key/Value pair, it sends the data to the next function
 
 
 - Function #2 <code>def fix_value(key, value)</code> takes the Key/Value pair and determines if it is an attribute that needs to be looked at based Key attribute. If it's not a value we are concerned with, it passes the value back to the first function. However, if it is a value that we are concerned with it first looks to see if the value even needs fixing. If yes, then the function fixes the value and returns the fixed value. If no, the function simply returns the value.
 
     
As we saw while auditing the data, phone numbers seemed to be of concern as the data format was all over the place. To address this, I stripped the phone number down to it's bare parts (3 digit area code, 7 digit number) by removing all special characters, spaces, and leading 1's. Then from the bare number, built the format that I wanted to see (ex: 2223334444 -> +1-222-333-4444). Code snippet below:

<code>elif 'phone' in key:
        phone_type_re = re.compile(r'^\+\d{1}[\-]\d{3}[\-]\d{3}[\-]\d{4}$')
        m = phone_type_re.search(value)
        if not m:
            remove_spec_char = ''.join(e for e in value if e.isalnum())
            remove_lead_one = remove_spec_char[1:]
            fixed_phone = '+1-' + remove_lead_one[:3] + '-' + remove_lead_one[3:6] + '-' + remove_lead_one[6:]
            return fixed_phone
        else:
            return value
</code>

After testing and validating the csv file creation and value fixing logic, I ran the script against the large OSM file to create individual csv files and prepare them for import into a MySQL database.

**Full code can be found in the file <code>../scripts/shape_data.py</code>**

**CSV files can be found in the <code>../data</code> folder**


### Analyzing the Data using SQL

Now that the data from the csv files have been loaded in to a SQL database we can explore our data. I have chosen to use a MySQL database and MySQL Workbench fo this activity.